In [1]:
import importlib
import utils
importlib.reload(utils)
from utils import *
from os import listdir
from os.path import join,basename

### Helper codes

In [ ]:
def computeAvg(x,kernel,boundaryCondition='periodical'):
    """
    Return the local average matrix
    """
    if boundaryCondition == 'periodical':
        nrow,ncol = x.shape[0],x.shape[1]
        x1 = np.zeros((nrow+2,ncol+2))
        x1[1:-1,1:-1] = x
        x1[0,1:-1] = x[-1,:]
        x1[-1,1:-1] = x[0,:]
        x1[1:-1,0] = x[:,-1]
        x1[1:-1,-1] = x[:,0]

        x1[0,0] = x1[0,-2]
        x1[0,-1] = x1[0,1]
        x1[-1,-1] = x1[-1,1]
        x1[-1,0] = x1[-1,-2]
    
    else:
        x1 = np.zeros((nrow+2,ncol+2))
        x1[1:-1,1:-1] = x
        x1[0,1:-1] = x[0,:]
        x1[-1,1:-1] = x[-1,:]
        x1[1:-1,0] = x[:,0]
        x1[1:-1,-1] = x[:,-1]

        x1[0,0] = x1[0,1]
        x1[0,-1] = x1[0,-2]
        x1[-1,-1] = x1[-1,-2]
        x1[-1,0] = x1[-1,1]
    
    xAvg = scipy.signal.convolve2d(x1,kernel,mode='same')
    xAvg = xAvg[1:-1,1:-1]
    return xAvg
        

In [ ]:
def computeCurl(uv):
    u = uv[:,:,0]
    v = uv[:,:,1]

    # simple kernel
    nrows,ncols = u.shape[0], u.shape[1]
    # add boundary
    u1 = np.zeros((nrows+2,ncols))
    u1[0,:] = u[0,:]
    u1[-1,:] = u[-1,:]
    u1_copy = np.zeros((nrows+2,ncols+2))
    u1_copy[:,1:-1] = u1
    u1_copy[:,0] = u1[:,1]
    u1_copy[:,-1] = u1[:,-1]
    # add boundary
    v1 = np.zeros((nrows+2,ncols))
    v1[0,:] = v[0,:]
    v1[-1,:] = v[-1,:]
    v1_copy = np.zeros((nrows+2,ncols+2))
    v1_copy[:,1:-1] = v1
    v1_copy[:,0] = v1[:,1]
    v1_copy[:,-1] = v1[:,-1]

    kernel = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])*1/8 #used to detect vertical lines
    uy = scipy.signal.convolve2d(u,kernel.T,mode='same') #detect horizontal lines for x velocity
    vx = scipy.signal.convolve2d(v,kernel,mode='same') #detect vertical lines for y velocity
    uy = uy[1:-1,1:-1]
    vx = vx[1:-1,1:-1]
    vort = vx - uy

    # complex kernel
    h = np.array([-1, 9, -45, 0, 45, -9, 1])/60;        # derivative used by Bruhn et al "combing "IJCV05' page218
    vx = scipy.ndimage.correlate(v, h, mode='nearest').transpose() #MATLAB returns transpose of the expected answer
    uy = scipy.ndimage.correlate(u, h.T, mode='nearest').transpose()
    vort = vx - uy
    return vort


In [ ]:
def HS_estimation(im1,im2,uLast,vLast,lamb=1,iter=200,boundaryCondition='periodical',estimation_method='HS'):
    """
    Adapted from Shengze Cai
    https://github.com/shengzesnail/coarse_to_fine_HS_PIV/blob/master/HS_Estimation.m
    im1,im2 (np.array): two subsequent frames or images.
    lamb (float): lambda, a parameter that reflects the influence of the smoothness term.
    iter (int): number of iterations.
    uLast, vLast (float): the flow field estimates of last pyramid; default is zero.
    estimationMethod (str): either HS or TE (where transport equation is considered in the constraint)
    """
    uInitial = uLast
    vInitial = vLast
    u = uInitial
    v = vInitial

    # Estimate Spatiotemporal derivatives of images
    fx, fy, ft, fxy = computeDerivatives_f(im1, im2, boundaryCondition)

    # averaging kernel
    kernel_1=[[0, 1/4, 0],[1/4, 0 ,1/4],[0, 1/4, 0]]

    for i in range(iter):
        # Compute local averages of the flow vectors
        uAvg = computeAvg(u,kernel_1,boundaryCondition)
        vAvg = computeAvg(v,kernel_1,boundaryCondition)
        if estimation_method == 'HS':
            Diffu = 0
        else:
            Diffu = -(1/Re/Sc)*fxy
        
        #Compute flow vectors constrained by its local average and the optical flow constraints
        data = ( fx * (uAvg-uLast) ) + ( fy * (vAvg-vLast) ) + ft + Diffu
        u = uAvg - ( fx * ( data ) ) / ( lamb + fx**2 + fy**2)
        v = vAvg - ( fy * ( data ) )/ ( lamb + fx**2 + fy**2)
    
    return u,v




In [ ]:
def HS_pyramids(im1,im2,lamb,PARA,uvInitial):
    """
    Horn-Schunck Motion Estimation Using Multi-Pyramids (Multi-Resolution)
    Ref:	
    Ruhnau, P., et al. (2005) Experiments in Fluids 38(1):21-32.
    Heitz, D., et al. (2010) Experiments in Fluids, 48(3):369-393.
    Sun, D., et al. (2010). Computer Vision & Pattern Recognition.

    Usage: [u, v] = HS_Pyramids(img1,img2,lambda,PARA)
    ********** inputs ***********
    im1,im2 (np.array): two subsequent frames or images (greyscale).
    lambda (float): a parameter that reflects the influence of the smoothness term.
    PARA: parameters
    ********** outputs ************
    u, v: the velocity components

    Shengze Cai, 2016/03
    """
    # initialise the velocity field
    if uvInitial is None:
        uInitial = np.zeros(im1.shape)
        vInitial = np.zeros(im1.shape)
        uvInitial = np.stack([uInitial,vInitial],axis=2)

    

    def Estimate(im1,im2,lamb,uvInitial,PARA):
        # Run Horn_schunck on all levels and interpolate
        warp_iter = PARA.warp_iter
        sizeOfMF = PARA.sizeOfMF
        isMedianFilter = PARA.isMedianFilter
        Dx = uvInitial[:,:,0] #delta x
        Dy = uvInitial[:,:,1] #de;ta y

        #consruct image pyramid for gnc stage 1
        pyramid_level = PARA.pyramid_level
        G1 = image_pyramid(im1,pyramid_level)
        G2 = image_pyramid(im2,pyramid_level)
        # iteration
        level = pyramid_level
        for l in range(level+1):
            small_im1 = G1[l]
            small_im2 = G2[l]
            sz = small_im1.shape
            uv = resample_flow(np.stack([Dx,Dy],2),sz)
            Dx = uv[:,:,0]
            Dy = uv[:,:,1]

            for iwarp in range(warp_iter):
                W1 = warp_forward(small_im1,Dx,Dy,PARA.interpolation_method)
                W2 = warp_inverse(small_im2, Dx,Dy,PARA.interpolation_method)
                Dx,Dy = HS_estimation(W1,W2,lamb, PARA.iter,Dx,Dy,PARA.boundaryCondition)

                if (isMedianFilter is True):
                    Dx = scipy.ndimage.median_filter(Dx,sizeOfMF)
                    Dy = scipy.ndimage.median_filter(Dy,sizeOfMF)

        return Dx, Dy
    
    # Run HS with multi-pyramids
    u, v = Estimate(im1, im2, lamb, uvInitial,PARA)

    return u,v
    
    

In [ ]:
class PARA:
    """
    parameters settings
    """
    def __init__(self,pyramid_level,warp_iter,iter,boundaryCondition='periodical',interpolation_method='spline',isMedianFilter=True,sizeOfMF=(5,5)):
        self.pyramid_level = pyramid_level
        self.warp_iter = warp_iter
        self.iter = iter
        #Boundary conditions
        self.boundaryCondition = boundaryCondition
        self.interpolation_method = interpolation_method
        #Divergence_free decomposition and the settings
        self.isMedianFilter = isMedianFilter
        self.sizeOfMF = sizeOfMF
